# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Следует обучить модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

Необходимо построить модель со значением метрики качества ***F1* не меньше 0.75**. 

**План выполнения проекта**

1. Загрузить и подготовить данные.
2. Обучить разные модели. 
3. Сделать выводы.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак. 


## Подготовка

In [1]:
# импортирование всех нужных библиотек и функций 
import nltk
import numpy as np
import pandas as pd
import re
import warnings

from catboost import CatBoostClassifier
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 

from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split

# загрука необходимых пакетов
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

# игнорирование предупреждений 
warnings.filterwarnings('ignore') 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\whois\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\whois\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\whois\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\whois\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


### Обзор данных
По первому взгляду на данные можно сказать:

* Имеется не несущий информацию столбец 'Unnamed: 0', который скорее всего возник при сохранении и повторном чтении датафрейма, следует указать параметр `index_col` для корректного чтения.
* Пропущенные значения отсутствуют.
* Два возможных значения целевого признака дают понять, что перед нами задача бинарной классификации.

In [2]:
# формирование датафрейма
data = pd.read_csv('dataset_comments.csv', index_col=0)

# обзор данных 
display(data.head())
data.info()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.7+ MB


### Распределение целевого признака
Позитивных комментариев почти в 9 раз больше, чем негативных, это стоит учесть при разделении данных на выборки.

In [3]:
# количество объектов на каждое значение целевого признака
display(data['toxic'].value_counts())

0    143346
1     16225
Name: toxic, dtype: int64

### Очистка текста

* Удалим лишние символы.
* Заменим ссылки и числа на 'URL' и 'NUM' соответственно.
* Удалим лишние пробелы.
* Очистим текст от стоп-слов.

In [4]:
# список с английскими стоп-словами
stop_words = stopwords.words('english')

# функция по очистке текста
def clean_text(text):
    # привидение текста к нижнему регистру
    text = text.lower()
    # удаление лишних символов 
    text = re.sub(r'[\*+\#+\№\"\-+\+\=+\?+\&\^\.+\;\,+\>+\(\)\/+\:\\+]', '', text)
    # замена ссылок на 'URL'
    text = re.sub(r'(http\S+)|(www\S+)|([\w\d]+www\S+)|([\w\d]+http\S+)', r'URL', text)
    # замена чисел на 'NUM'
    text = re.sub(r'(\d+\s\d+)|(\d+)',' NUM ', text)
    # удаление лишних пробелов
    text = re.sub(r'\s+', ' ', text)
    # очистка текста от стоп-слов
    text = word_tokenize(text)
    text = [word for word in text if word not in stop_words]
    
    return ' '.join(text)

In [5]:
# создание нового столбца с очищенным текстом
data['lemm_text'] = data['text'].apply(clean_text)

### Лемматизация текста 

In [6]:
# перевод объекта Series в список 
corpus = data['lemm_text'].tolist()

# пустой список для лемматизированного текста
lemm_corpus = []

# лемматизатор
wnl = WordNetLemmatizer()

# цикл лемматизации текста 
for i in range(len(corpus)):
    corpus[i] = corpus[i].split()
    lemm_list = []
    for j in range(len(corpus[i])):
        lemm_list.append(wnl.lemmatize(corpus[i][j]))
    lemm_list = ' '.join(lemm_list)
    lemm_corpus.append(lemm_list)

# добавление столбца с лемматизированным текстом 
data['lemm_text'] = lemm_corpus

### Разделение данных

Разделение данных на обучающую, валидационную и тестовую выборки в соотношении 3:1:1 соответственно, с учетом дисбаланса классов.

In [7]:
# формирование обучающей, валидационной и тестовой выборок
data_train, data_temporary = train_test_split(data, test_size=0.4, random_state=42, stratify=data['toxic'])
data_valid, data_test = train_test_split(data_temporary, test_size=0.5, random_state=42, stratify=data_temporary['toxic'])

# проверка соотношения размеров выборок
display(data_train.shape[0], data_valid.shape[0], data_test.shape[0])

# счетчик величин TI-IDF с учетом стоп-слов
count_tf_idf = TfidfVectorizer(stop_words=stop_words)

# признаки и целевой признак в обучающей выборке 
features_train = count_tf_idf.fit_transform(data_train['lemm_text'])
target_train = data_train['toxic']

# признаки и целевой признак в валидационной выборке 
features_valid = count_tf_idf.transform(data_valid['lemm_text'])
target_valid = data_valid['toxic']

# признаки и целевой признак в тестовой выборке 
features_test = count_tf_idf.transform(data_test['lemm_text'])
target_test = data_test['toxic']

95742

31914

31915

## Обучение
Рассмотрим три модели: 
* Логистическая регрессия.
* RandomForestClassifier.
* CatBoostClassifier.

### Функция нахождения наилучшего порога вероятности

In [8]:
# функция для нахождения наилучшего порога, возвращает порог, f1-меру с заданным порогом и матрицу ошибок
def best_threshold(probabilities_one, target):
    max_f1_score = 0
    best_threshold = 0
    for threshold in np.arange(0, 0.8, 0.02):
        predicted = probabilities_one > threshold
        f1 = f1_score(predicted, target)
        if max_f1_score < f1:
            max_f1_score = f1
            best_threshold = threshold
            matrix = confusion_matrix(target, predicted)
    return (f"Значение F1-меры = {max_f1_score:.3f}, при пороге вероятности: {best_threshold}", matrix)

### Логистическая регрессия
Показатели модели без подбора порога:
* **F1-мера: 0.739.**

Показатели модели с подобранным порогом:
* **F1-мера: 0.781.**
* Порог вероятности: 0.72.

Стоит отметить, что модель с порогом вероятности 0.72 показывает F1-меру выше, нежели модель со стандартным значением порога, но при этом она дает **меньше ложно-положительных ответов и больше ложно-отрицательных ответов**.

In [25]:
# вариации гиперпараметров
fit_intercept = [True, False]
class_weight = ['balanced']

# сетка гиперпараметров
param_grid = {'fit_intercept':fit_intercept,
              'class_weight':class_weight}

# модель 'логистическая регрессия'
model_log_reg = GridSearchCV(LogisticRegression(random_state=42), param_grid=param_grid, scoring='f1', cv=5)

In [26]:
%%time 

# обучение модели
model_log_reg.fit(features_train, target_train)

# предсказания модели
pred_valid = model_log_reg.predict(features_valid)

# значение F1-меры и матрица ошибок без подбора порога 
display(f"Значение F1-меры = {f1_score(target_valid, pred_valid):.3f}")
display(confusion_matrix(target_valid, pred_valid))

'Значение F1-меры = 0.739'

array([[27238,  1431],
       [  507,  2738]], dtype=int64)

CPU times: total: 16.3 s
Wall time: 15.3 s


In [27]:
%%time 

# вероятность положительного значения целевого признака
probabilities_one_valid = model_log_reg.predict_proba(features_valid)[:, 1]

# максимальное значение F1-меры с учетом нахождения порога вероятности
display(best_threshold(probabilities_one_valid, target_valid))

('Значение F1-меры = 0.781, при пороге вероятности: 0.72',
 array([[28161,   508],
        [  839,  2406]], dtype=int64))

CPU times: total: 2 s
Wall time: 1.97 s


### RandomForestClassifier
Показатели модели без подбора порога:
* **F1-мера: 0.495.**

Показатели модели с подобранным порогом:
* **F1-мера: 0.659.**
* Порог вероятности: 0.54.

Модель 'RandomForestClassifier' с подобранным порогом вероятности показывает F1-меру выше, нежели модель со стандартным значением порога, но при этом она дает **меньше ложно-положительных ответов и больше ложно-отрицательных ответов**, как и модель 'логистическая регрессия'.

In [106]:
# вариации гиперпараметров
max_depth = [int(x) for x in np.linspace(10, 50, num = 3)]
n_estimators = [int(x) for x in np.linspace(10, 100, num = 3)]
class_weight = ['balanced']

# сетка гиперпараметров
param_grid = {'max_depth':max_depth,
              'n_estimators':n_estimators,
              'class_weight':class_weight}

# модель 'случайный лес'
model_forest = GridSearchCV(RandomForestClassifier(random_state=42), param_grid=param_grid, scoring='f1', cv=2)

In [107]:
%%time

# обучение модели
model_forest.fit(features_train, target_train)

# предсказания модели на валидационной выборке
pred_valid = model_forest.predict(features_valid)

# значение F1-меры и матрица ошибок без подбора порога 
display(f"Значение F1-меры = {f1_score(target_valid, pred_valid):.3f}")
display(confusion_matrix(target_valid, pred_valid))

'Значение F1-меры = 0.495'

array([[24280,  4389],
       [  736,  2509]], dtype=int64)

CPU times: total: 5min 5s
Wall time: 5min 5s


In [108]:
%%time

# вероятность положительного значения целевого признака
probabilities_one_valid = model_forest.predict_proba(features_valid)[:, 1]

# максимальное значение F1-меры с учетом нахождения порога вероятности
display(best_threshold(probabilities_one_valid, target_valid))

('Значение F1-меры = 0.659, при пороге вероятности: 0.54',
 array([[27866,   803],
        [ 1256,  1989]], dtype=int64))

CPU times: total: 2.25 s
Wall time: 2.24 s


### CatBoostClassifier
Показатели модели без подбора порога:
* **F1-мера: 0.752.**

Показатели модели с подобранным порогом:
* **F1-мера: 0.777.**
* Порог вероятности: 0.66.

Аналогичный результат и у модели 'CatBoostClassifier': с подобранным порогом вероятности показывает F1-меру выше, нежели модель со стандартным значением порога, но при этом она дает **меньше ложно-положительных ответов и больше ложно-отрицательных ответов**.

In [109]:
%%time

# модель 'CatBoostClassifier'
model_cat = CatBoostClassifier(random_state=42, iterations=1000, auto_class_weights='Balanced', 
                               eval_metric='F1', verbose=100, loss_function='Logloss')
# обучение модели
model_cat.fit(features_train, target_train)

# предсказания модели на валидационной выборке
pred_valid = model_cat.predict(features_valid)

# значение F1-меры и матрица ошибок без подбора порога 
display(f"Значение F1-меры = {f1_score(target_valid, pred_valid):.3f}")
display(confusion_matrix(target_valid, pred_valid))

Learning rate set to 0.072254
0:	learn: 0.5128754	total: 590ms	remaining: 9m 49s
100:	learn: 0.8264020	total: 50.4s	remaining: 7m 28s
200:	learn: 0.8643951	total: 1m 39s	remaining: 6m 33s
300:	learn: 0.8837338	total: 2m 27s	remaining: 5m 43s
400:	learn: 0.8987199	total: 3m 16s	remaining: 4m 52s
500:	learn: 0.9089234	total: 4m 4s	remaining: 4m 3s
600:	learn: 0.9185545	total: 4m 52s	remaining: 3m 14s
700:	learn: 0.9248350	total: 5m 40s	remaining: 2m 25s
800:	learn: 0.9320198	total: 6m 28s	remaining: 1m 36s
900:	learn: 0.9373197	total: 7m 16s	remaining: 47.9s
999:	learn: 0.9419959	total: 8m 3s	remaining: 0us


'Значение F1-меры = 0.752'

array([[27471,  1198],
       [  567,  2678]], dtype=int64)

CPU times: total: 1h 1min 51s
Wall time: 8min 8s


In [110]:
%%time

# вероятность положительного значения целевого признака
probabilities_one_valid = model_cat.predict_proba(features_valid)[:, 1]

# максимальное значение F1-меры с учетом нахождения порога вероятности
display(best_threshold(probabilities_one_valid, target_valid))

('Значение F1-меры = 0.777, при пороге вероятности: 0.66',
 array([[28080,   589],
        [  807,  2438]], dtype=int64))

CPU times: total: 2.95 s
Wall time: 2.18 s


### Сравнение моделей
По совокупным результам наилучшие показатели имеет модель 'логистическая регрессия' с подобранным порогом вероятности. <br>
**F1-мера модели на валидационных данных: 0.781.**

In [119]:
# датафрейм с результатами моделей 
models_stats = pd.DataFrame(data={'F1_score':[.739, .495, .752], 
                                  'F1_score_with_threshold':[.781, .659, .777],
                                  'threshold':[.72, .54, .66]}, 
                            index=['LogisticRegression', 'RandomForestClassifier', 'CatBoostClassifier'])
display(models_stats)

,F1_score,F1_score_with_threshold,threshold
LogisticRegression,0.739,0.781,0.72
RandomForestClassifier,0.495,0.659,0.54
CatBoostClassifier,0.752,0.777,0.66


### Сравнение с константной моделью

Для проверки модели на адекватность сравним ее с результами константной модели. <br>
**F1-мера константной модели: 0.171**, выбранная модель показывает результаты лучше, следовательно модель адекватна.

In [111]:
# константная модель 
model_dummy = DummyClassifier(strategy='uniform', random_state=42)

# обучение константной модели
model_dummy.fit(features_train, target_train)

# F1-мера константной модели
display(f1_score(target_valid, model_dummy.predict(features_valid)))

0.17125605122065482

### Тестирование модели

In [120]:
# предсказания выбранной модели на тестовой выборке
pred_test = model_log_reg.predict(features_test)

# вероятность положительного значения целевого признака
proba_one_test = model_log_reg.predict_proba(features_test)[:, 1]

# установка порога
pred_test = proba_one_test > .72

# F1-мера выбранной модели на тестовой выборке
display(f1_score(target_test, pred_test))

0.7737630208333334

## Выводы

* Провели первичный обзор данных.
* Проанализировали распределение целевого признака.
* Очистили текст от лишних символов и пробелов, заменили ссылки и числа на 'URL' и 'NUM' соответственно, очистили текст от стоп-слов.
* Лемматизировали текст.
* Разделили данные на обучающую, валидационную и тестовую выборки в соотношении 3:1:1 соответственно, учли при этом дисбаланс классов.
* Обучили и сравнили между собой три разные модели: 'логистическая регрессия', 'RandomForestClassifier', 'CatBoostClassifier'. Подобрали наилучшую модель по F1-мере с подобранным порогом вероятности классов - 'логистическая регрессия'.
* Провели проверку выбранной модели на адекватность, путем сравнения ее с показателями константной модели.
* Протестировали выбранную модель на тестовой выборке. **Конечная F1-мера модели на тестовых данных: 0.773**